In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_excel('/content/drive/MyDrive/NNFL-Assignment2/data5.xlsx',header=None)
data

,0,1,2,3,4,5,6,7
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1
...,...,...,...,...,...,...,...,...
205,12.19,13.20,0.8783,5.137,2.981,3.631,4.870,3
206,11.23,12.88,0.8511,5.140,2.795,4.325,5.003,3
207,13.20,13.66,0.8883,5.236,3.232,8.315,5.056,3
208,11.84,13.21,0.8521,5.175,2.836,3.598,5.044,3


In [4]:
data = data.drop_duplicates(keep='first')
data = data.sample(frac = 1).reset_index(drop=True)

In [5]:
datan = data.values

X = datan[:,0:6]
m = X.shape[0] 

X = (X- np.min(X, axis = 0))/(np.max(X, axis = 0)- np.min(X, axis = 0)) 
ones = np.ones([m, 1]) 
X = np.append(ones,X, axis=1) 

y=datan[:,-1:] 

In [6]:
X_train_val, xtest, y_train_val, ytest = train_test_split(X, y, test_size=0.2)
xtrain, xvali, ytrain, yvali = train_test_split(X_train_val, y_train_val, test_size=0.1)
print(xtrain.shape, xvali.shape, xtest.shape, ytrain.shape, yvali.shape, ytest.shape)

xtrain_size=int(xtrain.size/8)
xvali_size=int(xvali.size/8)
xtest_size=int(xtest.size/8)


Ytr=np.zeros((xtrain_size,3))
for i in range(xtrain_size):
  if(ytrain[i]==1):
    Ytr[i]=[1,0,0]
  elif(ytrain[i]==2):
    Ytr[i]=[0,1,0]
  else:
    Ytr[i]=[0,0,1]

Yvali=np.zeros((xvali_size,3))
for i in range(xvali_size):
  if(yvali[i]==1):
    Yvali[i]=[1,0,0]
  elif(yvali[i]==2):
    Yvali[i]=[0,1,0]
  else:
    Yvali[i]=[0,0,1]

Ytest=np.zeros((xtest_size,3))
for i in range(xtest_size):
  if(ytest[i]==1):
    Ytest[i]=[1,0,0]
  elif(ytest[i]==2):
    Ytest[i]=[0,1,0]
  else:
    Ytest[i]=[0,0,1]

(151, 7) (17, 7) (42, 7) (151, 1) (17, 1) (42, 1)


In [7]:
def KMeans(K, data, n_iter, max_steps):
    best_cluster_centers = np.zeros((K, data.shape[1])) 
    min_dist = np.inf 
    
    for i in range(n_iter):
        init = np.random.choice(np.arange(data.shape[0]), K, replace = False) 
        centroids = data[init]      
        for j in range(max_steps):
            assigned_clusters = assign_clusters(data, centroids)
            
            
            if(len(np.unique(assigned_clusters)) != K):
                break
            
            prev_cluster_centers = centroids 
            for k in range(K):
                centroids[k] = np.average(data[assigned_clusters == k], axis = 0)
            
            
            if(np.sum(centroids == prev_cluster_centers) == np.size(centroids)):
                break 
        
        distance = dist_1(data, centroids)
        if(distance < min_dist):        
            best_cluster_centers = centroids
            labels = assigned_clusters
            min_dist = distance
    
    
    return best_cluster_centers,labels
    

def assign_clusters(data, centroids):
    K = centroids.shape[0]
    distances = np.zeros((data.shape[0], K))
    
    for k in range(K):
        distances[:, k] = np.sqrt(np.sum((data - centroids[k])**2, axis = 1))
    
    assigned_clusters = np.argmin(distances, axis = 1)
    return assigned_clusters

def dist_1(data, centroids):
    assigned_clusters = assign_clusters(data, centroids)
    dist = 0
    for k in range(centroids.shape[0]):
        dist += np.sum((data[assigned_clusters == k] - centroids[k])**2)
    
    return dist

**Gaussian Kernel**

In [8]:
alpha_vals=np.linspace(2,10,10)
vali_mat_lst=[]
test_mat_lst=[]
weight_lst=[]
for alpha in alpha_vals:
  k=int(xtrain_size/(alpha*10))
  k_means=KMeans(k,xtrain, 50, 100)
  beta=np.zeros((k))
  htrain_mat=np.zeros((xtrain_size,k))
  hvali_mat=np.zeros((xvali_size,k))
  htest_mat=np.zeros((xtest_size,k))
  for i in range(k):
    count=0
    sum1=0.0
    for j in range(xtrain_size):
      if(k_means[1][j]==i):
        sum1=sum1+((np.sum((xtrain[i]-k_means[0][i])**2))**0.5)
        count=count+1
    sigma=sum1/count
    beta[i]=1/(2*sigma*sigma)   
  for i in range(xtrain_size):
    for j in range(k):
      htrain_mat[i][j]=math.exp(-1*beta[j]*(np.sum(xtrain[i]-k_means[0][j])**2))
  for i in range(xvali_size):
    for j in range(k):
      hvali_mat[i][j]=math.exp(-1*beta[j]*(np.sum(xvali[i]-k_means[0][j])**2))
  for i in range(xtest_size):
    for j in range(k):
      htest_mat[i][j]=math.exp(-1*beta[j]*(np.sum(xtest[i]-k_means[0][j])**2))        
  weight_mat=np.dot(np.linalg.pinv(htrain_mat),Ytr)
  test_mat_lst.append(htest_mat)
  vali_mat_lst.append(hvali_mat)
  weight_lst.append(weight_mat)

In [9]:
acc_lst=[]
for i in range(len(alpha_vals)):
  ypred=np.dot(vali_mat_lst[i],weight_lst[i])
  yvali_pred=np.argmax(ypred,axis=1)
  count=0
  for j in range(xvali_size):
    if(yvali[i]==(yvali_pred[i]+1)):
      count=count+1
  accuracy=count/xvali_size
  acc_lst.append(accuracy)     
    

In [10]:
index=acc_lst.index(max(acc_lst))

ypred=np.dot(test_mat_lst[index],weight_lst[index])
ypred=np.argmax(ypred,axis=1) + 1

In [11]:
def confusion_matrix(predictions, labels):
  conf_matrix = [0,0,0,0,0,0,0,0,0]
  m = predictions.shape[0]
  for i in range(m):
    conf_matrix[3*(labels[i][0]-1) + predictions[i] - 1] = conf_matrix[3*(labels[i][0]-1) + predictions[i] - 1] + 1 
  return conf_matrix

In [12]:
conf_matrix = confusion_matrix(ypred.astype(int),ytest.astype(int))
ind_acc1=conf_matrix[0]/sum(conf_matrix[0:3])
ind_acc2=conf_matrix[4]/sum(conf_matrix[3:6])
ind_acc3=conf_matrix[8]/sum(conf_matrix[6:9]) 
overall=(conf_matrix[0] + conf_matrix[4] + conf_matrix[8])/sum(conf_matrix)

In [13]:
print("Individual accuracy 1: ",ind_acc1)
print("Individual accuracy 2: ",ind_acc2)
print("Individual accuracy 3: ",ind_acc3)
print("Overall accuracy : ", overall)

Individual accuracy 1:  0.7777777777777778
Individual accuracy 2:  1.0
Individual accuracy 3:  0.9230769230769231
Overall accuracy :  0.9166666666666666


**Linear Kernel**

In [14]:
alpha_vals=np.linspace(2,10,10)
vali_mat_lst=[]
test_mat_lst=[]
weight_lst=[]
for alpha in alpha_vals:
  k=int(xtrain_size/(alpha*10))
  k_means=KMeans(k,xtrain, 50, 100)
  beta=np.zeros((k))
  htrain_mat=np.zeros((xtrain_size,k))
  hvali_mat=np.zeros((xvali_size,k))
  htest_mat=np.zeros((xtest_size,k)) 
  for i in range(xtrain_size):
    for j in range(k):
      htrain_mat[i][j]=(np.sum(xtrain[i]-k_means[0][j])**2)**0.5
  for i in range(xvali_size):
    for j in range(k):
      hvali_mat[i][j]=(np.sum(xvali[i]-k_means[0][j])**2)**0.5
  for i in range(xtest_size):
    for j in range(k):
      htest_mat[i][j]=(np.sum(xtest[i]-k_means[0][j])**2)**0.5        
  weight_mat=np.dot(np.linalg.pinv(htrain_mat),Ytr)
  test_mat_lst.append(htest_mat)
  vali_mat_lst.append(hvali_mat)
  weight_lst.append(weight_mat)

In [15]:
acc_lst=[]
for i in range(len(alpha_vals)):
  ypred=np.dot(vali_mat_lst[i],weight_lst[i])
  yvali_pred=np.argmax(ypred,axis=1)
  count=0
  for j in range(xvali_size):
    if(yvali[i]==(yvali_pred[i]+1)):
      count=count+1
  accuracy=count/xvali_size
  acc_lst.append(accuracy)     
    

In [16]:
index=acc_lst.index(max(acc_lst))
ypred=np.dot(test_mat_lst[index],weight_lst[index])
ypred=np.argmax(ypred,axis=1) + 1

In [17]:
conf_matrix = confusion_matrix(ypred.astype(int),ytest.astype(int))
ind_acc1=conf_matrix[0]/sum(conf_matrix[0:3])
ind_acc2=conf_matrix[4]/sum(conf_matrix[3:6])
ind_acc3=conf_matrix[8]/sum(conf_matrix[6:9]) 
overall=(conf_matrix[0] + conf_matrix[4] + conf_matrix[8])/sum(conf_matrix)

In [18]:
print("Individual accuracy 1: ",ind_acc1)
print("Individual accuracy 2: ",ind_acc2)
print("Individual accuracy 3: ",ind_acc3)
print("Overall accuracy : ", overall)

Individual accuracy 1:  0.7777777777777778
Individual accuracy 2:  1.0
Individual accuracy 3:  0.9230769230769231
Overall accuracy :  0.9166666666666666


**MULTI QUADRIC**

In [19]:
alpha_vals=np.linspace(2,10,10)
vali_mat_lst=[]
test_mat_lst=[]
weight_lst=[]
for alpha in alpha_vals:
  k=int(xtrain_size/(alpha*10))
  k_means=KMeans(k,xtrain, 50, 100)
  sigma1=np.zeros((k))
  htrain_mat=np.zeros((xtrain_size,k))
  hvali_mat=np.zeros((xvali_size,k))
  htest_mat=np.zeros((xtest_size,k))
  for i in range(k):
    count=0
    sum1=0.0
    for j in range(xtrain_size):
      if(k_means[1][j]==i):
        sum1=sum1+((np.sum((xtrain[i]-k_means[0][i])**2))**0.5)
        count=count+1
    sigma=sum1/count
    sigma1[i]=sigma*sigma  
  for i in range(xtrain_size):
    for j in range(k):
      htrain_mat[i][j]=((np.sum(xtrain[i]-k_means[0][j])**2)+sigma1[j])**0.5
  for i in range(xvali_size):
    for j in range(k):
      hvali_mat[i][j]=((np.sum(xvali[i]-k_means[0][j])**2)+sigma1[j])**0.5
  for i in range(xtest_size):
    for j in range(k):
      htest_mat[i][j]=((np.sum(xtest[i]-k_means[0][j])**2)+sigma1[j])**0.5        
  weight_mat=np.dot(np.linalg.pinv(htrain_mat),Ytr)
  test_mat_lst.append(htest_mat)
  vali_mat_lst.append(hvali_mat)
  weight_lst.append(weight_mat)
  

In [20]:
acc_lst=[]
for i in range(len(alpha_vals)):
  ypred=np.dot(vali_mat_lst[i],weight_lst[i])
  yvali_pred=np.argmax(ypred,axis=1)
  count=0
  for j in range(xvali_size):
    if(yvali[i]==(yvali_pred[i]+1)):
      count=count+1
  accuracy=count/xvali_size
  acc_lst.append(accuracy)     
    

In [21]:
index=acc_lst.index(max(acc_lst))
ypred=np.dot(test_mat_lst[index],weight_lst[index])
ypred=np.argmax(ypred,axis=1)+1

conf_matrix = confusion_matrix(ypred.astype(int),ytest.astype(int))
ind_acc1=conf_matrix[0]/sum(conf_matrix[0:3])
ind_acc2=conf_matrix[4]/sum(conf_matrix[3:6])
ind_acc3=conf_matrix[8]/sum(conf_matrix[6:9]) 
overall=(conf_matrix[0] + conf_matrix[4] + conf_matrix[8])/sum(conf_matrix)

In [22]:
print("Individual accuracy 1: ",ind_acc1)
print("Individual accuracy 2: ",ind_acc2)
print("Individual accuracy 3: ",ind_acc3)
print("Overall accuracy : ", overall)

Individual accuracy 1:  0.7777777777777778
Individual accuracy 2:  0.9285714285714286
Individual accuracy 3:  0.9230769230769231
Overall accuracy :  0.8888888888888888


**GENERALIZED MULTIQUADRIC**

In [23]:
alpha_vals=np.linspace(2,10,10)
vali_mat_lst=[]
test_mat_lst=[]
weight_lst=[]
for alpha in alpha_vals:
  k=int(xtrain_size/(alpha*10))
  k_means=KMeans(k,xtrain, 50, 100)
  beta=np.zeros((k))
  sigma1=np.zeros((k))
  htrain_mat=np.zeros((xtrain_size,k))
  hvali_mat=np.zeros((xvali_size,k))
  htest_mat=np.zeros((xtest_size,k))
  for i in range(k):
    count=0
    sum1=0.0
    for j in range(xtrain_size):
      if(k_means[1][j]==i):
        sum1=sum1+((np.sum((xtrain[i]-k_means[0][i])**2))**0.5)
        count=count+1
    sigma=sum1/count
    sigma1[i]=sigma*sigma
    beta[i]=1/(2*sigma*sigma)  
  for i in range(xtrain_size):
    for j in range(k):
      htrain_mat[i][j]=((np.sum(xtrain[i]-k_means[0][j])**2)+sigma1[j])**beta[j]
  for i in range(xvali_size):
    for j in range(k):
      hvali_mat[i][j]=((np.sum(xvali[i]-k_means[0][j])**2)+sigma1[j])**beta[j]
  for i in range(xtest_size):
    for j in range(k):
      htest_mat[i][j]=((np.sum(xtest[i]-k_means[0][j])**2)+sigma1[j])**beta[j]        
  weight_mat=np.dot(np.linalg.pinv(htrain_mat),Ytr)
  test_mat_lst.append(htest_mat)
  vali_mat_lst.append(hvali_mat)
  weight_lst.append(weight_mat)
  

In [24]:
acc_lst=[]
for i in range(len(alpha_vals)):
  ypred=np.dot(vali_mat_lst[i],weight_lst[i])
  yvali_pred=np.argmax(ypred,axis=1)
  count=0
  for j in range(xvali_size):
    if(yvali[i]==(yvali_pred[i]+1)):
      count=count+1
  accuracy=count/xvali_size
  acc_lst.append(accuracy) 

In [25]:
index=acc_lst.index(max(acc_lst))
ypred=np.dot(test_mat_lst[index],weight_lst[index])
ypred=np.argmax(ypred,axis=1)+1

conf_matrix = confusion_matrix(ypred.astype(int),ytest.astype(int))
ind_acc1=conf_matrix[0]/sum(conf_matrix[0:3])
ind_acc2=conf_matrix[4]/sum(conf_matrix[3:6])
ind_acc3=conf_matrix[8]/sum(conf_matrix[6:9]) 
overall=(conf_matrix[0] + conf_matrix[4] + conf_matrix[8])/sum(conf_matrix)

In [26]:
print("Individual accuracy 1: ",ind_acc1)
print("Individual accuracy 2: ",ind_acc2)
print("Individual accuracy 3: ",ind_acc3)
print("Overall accuracy : ", overall)

Individual accuracy 1:  0.7777777777777778
Individual accuracy 2:  0.9285714285714286
Individual accuracy 3:  0.9230769230769231
Overall accuracy :  0.8888888888888888
